In [2]:
!pip install duckdb

  Obtaining dependency information for duckdb from https://files.pythonhosted.org/packages/99/61/1ef0d3d13908b5ae6637103e0dbeafc58028f662caa366ecd871154dbc9f/duckdb-0.10.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached duckdb-0.10.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (763 bytes)
Using cached duckdb-0.10.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.8 MB)

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [81]:
import duckdb
conn = duckdb.connect()

In [82]:
# create schema (到時候要換名字～)
# SQL statement to create a schema if it does not exist
sql_statement_create_schema = "CREATE SCHEMA IF NOT EXISTS de;"

# Connect to DuckDB
# conn = duckdb.connect()

# Create a schema
result = conn.execute(sql_statement_create_schema)

# Check if the schema has been created successfully
schemas = conn.execute("SELECT schema_name FROM information_schema.schemata;").fetchall()
print("Available schemas:", schemas)

# conn.close()

Available schemas: [('de',), ('information_schema',), ('main',), ('pg_catalog',), ('information_schema',), ('main',), ('pg_catalog',), ('information_schema',), ('main',), ('pg_catalog',)]


### create tables

In [83]:
import os

sql_statements = """

CREATE TABLE IF NOT EXISTS de.us_eco (
    date DATE PRIMARY KEY,
    gdp_value FLOAT,
    unemployment_rate FLOAT,
    inflation_rate FLOAT,
    income FLOAT
);

CREATE TABLE IF NOT EXISTS de.us_interest (
    date DATE,
    interest_rate_multi_family_estate FLOAT,
    interest_rate_commercial_estate FLOAT,
    interest_owner_occupied_estate FLOAT,
    PRIMARY KEY (date),
);

CREATE TABLE IF NOT EXISTS de.us_estate (
    date DATE,
    cpi FLOAT,
    permits FLOAT,
    home_sales FLOAT,
    price_index FLOAT,
    PRIMARY KEY (date),
);

"""
# FOREIGN KEY (date) REFERENCES de.us_eco(date)
try:
    # Execute the SQL statements
    conn.execute(sql_statements)
    # Commit the transaction
    conn.commit()

    # Check if tables have been created successfully
    tables = conn.execute("SELECT * FROM information_schema.tables WHERE table_schema = 'de';").fetchall()
    print("Tables in the 'de' schema:", tables)

except Exception as e:
    # Handle any errors that occurred and print them out
    print("An error occurred:", e)

#finally:
    # Close the connection whether or not an error occurred
    # conn.close()



Tables in the 'de' schema: [('memory', 'de', 'us_eco', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None), ('memory', 'de', 'us_estate', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None), ('memory', 'de', 'us_interest', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None)]


### Insert csv to DuckDB

In [84]:
sql_statements = """
COPY de.us_eco FROM './federal_eco/us_eco.csv';
"""

conn.execute(sql_statements)
conn.commit()

sql_statements = """
COPY de.us_estate FROM './federal_estate/us_estate.csv';
"""
conn.execute(sql_statements)
conn.commit()

sql_statements = """
COPY de.us_interest FROM './federal_interest/us_interest.csv';
"""
conn.execute(sql_statements)
conn.commit()


eco = conn.execute("SELECT * FROM de.us_eco").fetchall()
#print(eco)

[(datetime.date(1948, 7, 1), 279.1960144042969, 3.5999999046325684, None, None), (datetime.date(1983, 7, 1), 3689.178955078125, 9.399999618530273, None, 5647.0), (datetime.date(1987, 10, 1), 5007.994140625, 6.0, None, 6619.60009765625), (datetime.date(1979, 4, 1), 2591.2470703125, 5.800000190734863, None, 5156.10009765625), (datetime.date(1980, 7, 1), 2856.48291015625, 7.800000190734863, None, 5189.7998046875), (datetime.date(1982, 1, 1), 3274.302001953125, 8.600000381469727, None, 5384.7001953125), (datetime.date(2015, 4, 1), 18279.783203125, 5.400000095367432, 1.8600000143051147, 13842.0), (datetime.date(1950, 7, 1), 308.15301513671875, 5.0, None, None), (datetime.date(1981, 10, 1), 3280.818115234375, 7.900000095367432, None, 5382.2998046875), (datetime.date(1983, 4, 1), 3578.847900390625, 10.199999809265137, None, 5561.2998046875), (datetime.date(2009, 1, 1), 14430.90234375, 7.800000190734863, 0.6100000143051147, 12361.5), (datetime.date(1947, 1, 1), 243.16400146484375, None, None, 